In [ ]:
import sys
import os
import netCDF4
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import glob
import json
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs
np.set_printoptions(threshold=np.inf)

In [ ]:
file = 'data\C3S-LC-L4-LCCS-Map-300m-P1Y-2019-v2.1.1.nc'

In [ ]:
ds = xr.open_dataset(file)
ds

In [ ]:
# drop variables that are not needed: processed_flag, current_pixel_state, observation_count, change_count, crs, time_bounds
ds = ds.drop(['processed_flag', 'current_pixel_state', 'observation_count', 'change_count', 'crs', 'time_bounds'])

In [ ]:
# ds_coarse = ds.coarsen(lat=3, lon=3).mean()
# ds_coarse.to_netcdf("data\C3S-LC-L4-LCCS-Map-600m-P1Y-2019-v2.1.1.nc")

In [ ]:
central_lat = -33.9248685
central_lon = 18.4240553

lat_min = central_lat - 0.5
lat_max = central_lat + 0.5

lon_min = central_lon - 0.5
lon_max = central_lon + 0.5
filtered_ds = ds.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))

In [ ]:
df = filtered_ds.to_dataframe()
df
# df.info()

In [ ]:
agg_df = df.groupby([
    df.index.get_level_values(0),  # Timestamp
    (pd.Series(df.index.get_level_values(1)).round(2) // 0.3 * 0.3).values,
    (pd.Series(df.index.get_level_values(2)).round(2) // 0.3 * 0.3).values,
    df.index.get_level_values(3)  # The fourth level (bounds)
]).agg({'lccs_class': 'mean'})

# Reset the index for a clean dataframe
agg_df = agg_df.reset_index()

agg_df.rename(columns={
    'level_1': 'Latitude',
    'level_2': 'Longitude',
    'bounds': 'Bounds'
}, inplace=True)

agg_df.head()

In [ ]:
# Pivot the DataFrame to create a matrix for the heatmap
# heatmap_data = agg_df.pivot_table(index='level_1', columns='level_2', values='lccs_class', aggfunc='mean')

# Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(heatmap_data, cmap='viridis', cbar_kws={'label': 'lccs_class'})
# plt.title('lccs_class across Latitudes and Longitudes')
# plt.show()


In [ ]:
# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(agg_df.Longitude, agg_df.Latitude)]
geo_df = gpd.GeoDataFrame(agg_df, geometry=geometry)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Set up the map
ax.set_extent([agg_df.Longitude.min() - 0.5, agg_df.Longitude.max() + 0.5, 
               agg_df.Latitude.min() - 0.5, agg_df.Latitude.max() + 0.5])

# Add features for countries, coastlines, and borders
ax.coastlines(resolution='10m')
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAND, edgecolor='black')

# Plot data points with colormap
scatter = ax.scatter(geo_df.Longitude, geo_df.Latitude, c=geo_df.lccs_class, 
                     cmap='viridis', s=40, transform=ccrs.PlateCarree())
plt.colorbar(scatter, ax=ax, orientation='vertical', label='lccs_class')

plt.show()

In [ ]:
# df.to_csv('bhengazi.csv')